# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.metrics import  confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM disaster_resp_mes;", engine)
df.head()

OperationalError: ignored

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
schema = inspector.get_schema_names()[0]
colnames = []
table_name = inspector.get_table_names(schema=schema)[0]
for column in inspector.get_columns(table_name, schema=schema):
    colnames.append(column['name'])
target_colnames = colnames[4:]


In [ ]:
X = df['message'][:7000]
Y = df[target_colnames][:7000]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """
    INPUT:
    text - string
    OUTPUT:
    tokens - list of strings
    
    function takes raw text, removes punctuation signs, substitutes
    with spaces. Puts all characters in lower case, tokenizes text
    by words, removes stop words, lemmatizes, and returns list of tokens 
    """
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]

    return tokens

# 2a. Feature Research
Search for messages that contain the word "update", plot the number of times this word is used in the message to the number of times it relates to any of target variables

In [3]:
X

NameError: ignored

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
f1_scores = []
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred[ind]))
    #display_results(y_test.values[ind], y_pred[ind])
    f1_scores.append(f1_score(y_test.values[ind], y_pred[ind]))

Class - related
             precision    recall  f1-score   support

          0       1.00      0.97      0.98        31
          1       0.83      1.00      0.91         5

avg / total       0.98      0.97      0.97        36

Class - request
             precision    recall  f1-score   support

          0       0.97      0.97      0.97        33
          1       0.67      0.67      0.67         3

avg / total       0.94      0.94      0.94        36

Class - offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00        36

Class - aid_related
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        35
          1       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00        36

Class - medical_help
             precision    recall  f1-score   support

          0       1.00      0.94      0.97        36

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
print('Base Model\nMinimum f1 score - {}\nBest f1 score - {}\nMean f1 score - {}'.format(min(f1_scores), max(f1_scores), round(sum(f1_scores)/len(f1_scores), 3)))

Base Model
Minimum f1 score - 0.0
Best f1 score - 1.0
Mean f1 score - 0.397


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f27d68d4488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [ ]:
parameters = {'tfidf__use_idf': [True, False],
              'tfidf__norm': ['l1', 'l2', None],
              #'vect__ngram_range': ((1, 1), (1, 2)),
              'clf__estimator__n_estimators': [100, 150],
              'clf__estimator__max_depth': [5, 15]

    }

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
f1_scores = []
for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred[ind]))
    f1_scores.append(f1_score(y_test.values[ind], y_pred[ind]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.